Phase 2 - Utility notebook (copied from Databricks, not all code may work correctly in other IDEs).

This notebook defines functions used in other Phase 2 notebooks.

**Author:** Nate Bean

**Date created:** 08/24-- **Last modified:** 08/24

**Purpose:** A utility that contains functions used for transforming and sending Phase 2 syphilis FHIR data. 

**Schedule:** None

**Changelog:**
- 08/05/2024: Finished adding initial functions

In [0]:
import json #send_fhir_message
import requests #send_fhir_message
import re #qr function
import pandas as pd #
import time #send_fhir_message

**Resource creation functions:**
- These functions are used part of the overall workflow for converting data to FHIR. They create the actual structure of each resource. Functionalizing this code makes the main main script more readable. 
- More of the process could be included in these functions, but I didn't want to load fhir.resources as part of this notebook.

In [0]:
def create_patient_resource(data):
    pat = {
        'id': data.case_id.item(),
        'active': 'true',
        'gender': data.gender.item(),
        'communication': [{'language': {'coding': [{'code': data.language_code.item(),
                                                    'display': data.preferred_language.item(),
                                                    'system': 'urn:ietf:bcp:47'}]}, 
                           'preferred': 'True'}],
        'extension': [{
            "id" : "genderIdentity",
            'url': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-genderIdentity',
            'valueCodeableConcept': {
                'coding': [{
                    'system' : data.gender_system.item(),
                    'code' : data.gender_code.item(),
                    'display' : data.gender_display.item()
                }],
                'text': data.gender_display.item()
                                    }
                }]
            }
    
    return(pat)

In [0]:
def create_observation_resource(data):
    obs = {
        'id': data.case_id.item(),
        'status': 'registered',
        'subject': {'reference': 'Patient/patient-'+ data.case_id.item()},
        'code': {    
            'coding': [{
            'system': 'https://www.hennepin.us/residents/health-medical/public-health',
            'code': orientation[orientation.Display.isin([data.sexual_orientation.item()])].Code.item(),
            'display': data.sexual_orientation.item()}]}
        }
    
    return(obs)

In [0]:
def create_careplan_resource(data):
    cp = {
          'id': data.case_id,
          'status': 'active',
          'intent': 'order',
          'subject': {'reference': 'Patient/patient-'+ data.case_id},
          'activity': [{'detail': {'kind': 'MedicationRequest',
                                   'status': 'completed',
                                   'scheduledString': str(data.treatment_date),
                                   'description': data.treatment_regimen}}]
            }
    
    return(cp)

In [0]:
#May need to send up to 4 values for sex partner gender using the logic below.
def create_questionnaire_resource(data):

    #Splitting this here so I don't need to do more work earlier in the data pipeline.
    spg = data.sex_partner_gender.item()
    spg = re.split(";", spg)
    
    if len(spg) == 1:
        qr = {
            'id': data.case_id.item(),
            'status': 'completed',
            'subject': {'reference': 'Patient/patient-'+ data.case_id.item()},
            'item': [{'linkId': '/Epic_1',
                      'text': 'Sex partner gender in the last 12 months',
                      'answer': [
                        {
                            'valueCoding': {'system': 'https://www.hennepin.us/residents/health-medical/public-health',
                                            'code': spg[0],
                                            'display': spg[0]}
                        }
                    ],
                }
            ]}

    elif len(spg) == 2:
        qr = {
            'id': data.case_id.item(),
            'status': 'completed',
            'subject': {'reference': 'Patient/patient-'+ data.case_id.item()},
            'item': [{'linkId': '/Epic_1',
                      'text': 'Sex partner gender in the last 12 months',
                      'answer': [
                        {
                            'valueCoding': {'system': 'https://www.hennepin.us/residents/health-medical/public-health',
                                            'code': spg[0],
                                            'display': spg[0]}
                        },                       
                        {
                            'valueCoding': {'system': 'https://www.hennepin.us/residents/health-medical/public-health',
                                            'code': spg[1],
                                            'display': spg[1]} 
                        }
                    ],
                }
            ]}

    elif len(spg) == 3:
        qr = {
            'id': data.case_id.item(),
            'status': 'completed',
            'subject': {'reference': 'Patient/patient-'+ data.case_id.item()},
            'item': [{'linkId': '/Epic_1',
                      'text': 'Sex partner gender in the last 12 months',
                      'answer': [
                        {
                            'valueCoding': {'system': 'https://www.hennepin.us/residents/health-medical/public-health',
                                            'code': spg[0],
                                            'display': spg[0]}
                        },                       
                        {
                            'valueCoding': {'system': 'https://www.hennepin.us/residents/health-medical/public-health',
                                            'code': spg[1],
                                            'display': spg[1]}  
                        },                       
                        {
                            'valueCoding': {'system': 'https://www.hennepin.us/residents/health-medical/public-health',
                                            'code': spg[2],
                                            'display': spg[2]}  
                        }
                    ],
                }
            ]}

    elif len(spg) == 4:
        qr = {
            'id': data.case_id.item(),
            'status': 'completed',
            'subject': {'reference': 'Patient/patient-'+ data.case_id.item()},
            'item': [{'linkId': '/Epic_1',
                      'text': 'Sex partner gender in the last 12 months',
                      'answer': [
                        {
                            'valueCoding': {'system': 'https://www.hennepin.us/residents/health-medical/public-health',
                                            'code': spg[0],
                                            'display': spg[0]}
                        },                       
                        {
                            'valueCoding': {'system': 'https://www.hennepin.us/residents/health-medical/public-health',
                                            'code': spg[1],
                                            'display': spg[1]}  
                        },                       
                        {
                            'valueCoding': {'system': 'https://www.hennepin.us/residents/health-medical/public-health',
                                            'code': spg[2],
                                            'display': spg[2]}  
                        },                       
                        {
                            'valueCoding': {'system': 'https://www.hennepin.us/residents/health-medical/public-health',
                                            'code': spg[3],
                                            'display': spg[3]}  
                        }
                    ],
                }
            ]}
    return(qr)

**send_fhir_message() description**

After a patient bundle is created, the send_fhir_message() function will send it to MDH's API. It will send a PUT request containing the data to the MDH endpoint, and the bulk of the code is used to properly respond to potential responses received from the API. Anything but a 200 'ok' response indicates that the attempt to send data was not successful. The function may initially run into a Cloudfront security error (I think because it takes the API time to spin up?) but will resend the message a couple times before failing in this scenario. See documentation for more details about potential error messages.

In [0]:
#Currently we only have a single base url, which points to the Prod environment
def send_fhir_message(base_url, fhir_bundle, verbose = True):
    endpoint = f'{base_url}/api/resources/proxy/idepc/fhir/sti'
    api_key = dbutils.secrets.get(scope = "PH-PrdMDW-dbw-PrdMDW-kv-secret-scope", key = "MDH-DEX-apikey")
    headers = {'Authorization': f'API {api_key}', 'Content-Type': "text/plain"}

    #Send data. 
    #It seems like it takes a sec for the server to start when contacted. Data submitted right away will get a 200 Cloudfront error. Use loop to retry
    attempts = 0
    success = 0

    while attempts < 3 and success == 0:
        resp = requests.put(endpoint, data = fhir_bundle, headers = headers)
        print('Patient bundle ' + patid + ' sent to: ' + endpoint)

    #Response handling
    #Valid response should be 200 OK, but 200 ok may also be an error with Cloudfront security - differentiate below
        if resp.status_code == 200:
            try:
                resp_json = json.loads(resp.content) #security error is not in json so this will fail
                resp_code = json.loads(resp.content)['entry'][0]['resource']['response']['code'] #if successful, this should be 'ok'

                if resp_code == 'ok':
                    success = 1

                    if verbose == True:
                        print(str(resp.status_code) + " - " + resp_code)

                else: #technically this would something that's json, but not successful. Not sure if this would ever happen, but adding just in case.
                    attempts += 1

                    if attempts == 3:
                        print(resp.content)
                        raise ValueError('200 OK - Cloudfront Security (3/3 attempts failed)')

                    else:
                        print(f"Waiting for 60 seconds, then retrying... (attempt {attempts} of 3)")
                        time.sleep(60)
                        
            except: #the response can't be parsed as json
                attempts += 1

                if attempts == 3:
                        print(resp.content)
                        raise ValueError('200 OK - Cloudfront Security (3/3 attempts failed)')

                else:
                    print(f"Waiting for 60 seconds, then retrying... (attempt {attempts} of 3)")
                    time.sleep(60)

        #Handling other values is more straightforward - they're errors and should halt execution.
        #400 should either mean invalid JSON data or bad values in valid JSON. Identify using printed text
        elif resp.status_code == 400:
            diagnostics = json.loads(resp.content)['entry'][1]['resource']['issue'][0]['diagnostics']
            print(diagnostics)
            raise ValueError('400 Bad Request')

        #500 should capture invalid API key, DEX code error. Identify using printed text and documentation.
        elif resp.status_code == 500:
            print(resp.content)
            raise ValueError('500 Internal Server Error')
        
        #This will capture any errors not included in the documentation. May want to add them into this function afterward.
        else:
            print(resp.content)
            raise ValueError(resp.status_code + ' Error - see text to diagnoise.')